In [1]:
# -*- coding: utf-8 -*- 

import re 
from tqdm import tqdm 
import time
from datetime import datetime
import sqlite3
import sys
import os
import pandas as pd
import unify

In [5]:
def connect(file_path, primary, columns):
    con = sqlite3.connect(file_path)
    cur = con.cursor()
    cols = ", ".join([c + ' Varchar' for c in columns]) 
    cur.execute("create table meta ("+primary+" Varchar PRIMARY KEY, "+cols+" )")
    cur.execute("CREATE INDEX log on meta (id);")
    cur.execute("create table plain_texts (id Varchar(128) NOT NULL PRIMARY KEY, text Varchar NOT NULL);")
    cur.execute("create table tagged_texts (id Varchar(128) NOT NULL PRIMARY KEY, text Varchar NOT NULL );")
    con.commit()
    return con, cur

In [3]:
workdir = r'/home/tari/Загрузки/taiga/Arzamas'
filename = 'Arzamas.db'
file_path = os.path.join(workdir, filename)
metatablepath = os.path.join(workdir,'newmetadata.csv')
tagged = os.path.join(workdir,'texts_tagged')
plain = os.path.join(workdir,'texts')

meta = pd.read_csv(metatablepath, sep='\t', encoding='utf8')
meta = meta.fillna('')
meta.head()

,source,id,title,theme,author,textdate,tags,URL,Author_profession,About_author
0,Arzamas,400,Бессмысленный тест на знание Arzamas •,[],,30.01.2017,[],http://arzamas.academy/mag/400-two-years,,
1,Arzamas,299,Черновики Михаила Булгакова •,['Литература'],Мария Котова,14.06.2016,[],http://arzamas.academy/mag/299-bulgakov,Историк литературы,"Кандидат филологических наук, ведущий научный ..."
2,Arzamas,164,За что на самом деле сожгли Джордано Бруно? •,['История'],Ованес Акопян,05.11.2015,[],http://arzamas.academy/mag/164-bruno,Историк,"Кандидат исторических наук, специалист по исто..."
3,Arzamas,282,5 новых книг о литературе •,['Литература'],,05.04.2016,[],http://arzamas.academy/mag/282-essay,,
4,Arzamas,303,Зачем американцы красили лицо в черный •,"['Искусство', 'Антропология']",Феликс Сандалов,04.07.2016,[],http://arzamas.academy/mag/303-menestrels,Журналист,Автор книги «Формейшен: история одной сцены»


In [14]:
if not os.path.exists(filename):
    con, cur = connect(filename, meta.columns[1], [meta.columns[0]]+list(meta.columns[2:]))
else:
    con = sqlite3.connect(filename, meta.columns[1], [meta.columns[0]]+list(meta.columns[2:]))
    cur = con.cursor()

In [8]:
meta.iloc[6].to_dict()

{'About_author': 'Кандидат искусствоведения, доцент Санкт-Петербургского государственного университета промышленных технологий и\xa0дизайна, лектор Русского музея, автор книг «Всадники особого назначения» (2013), «Костюм и\xa0мода Российской империи. Эпоха Николая\xa0II» (2013), «Костюм и\xa0мода Российской империи. Эпоха Александра\xa0II и\xa0Александра\xa0III» (2015), «Молодые и\xa0красивые. Мода 1920-х годов» (2016).',
 'Author_profession': 'Искусствовед, историк моды',
 'URL': 'http://arzamas.academy/mag/208-russo-turkish-war',
 'author': 'Ольга Хорошилова',
 'id': 208,
 'source': 'Arzamas',
 'tags': '[]',
 'textdate': '07.12.2015',
 'theme': "['История', 'Антропология']",
 'title': 'Как Русско-турецкая война повлияла на моду  •  '}

In [9]:
#meta = meta.drop_duplicates(subset=None, keep='first', inplace=False)
meta.id.value_counts()[:10]

452    1
221    1
211    1
212    1
214    1
215    1
216    1
218    1
220    1
222    1
Name: id, dtype: int64

In [ ]:
for i in tqdm(range(len(meta))):
    values = meta.iloc[i].to_dict()    
    values['id'] = str(values['id'])
    columns = ', '.join(values.keys())
    
    #print(list(values.values()))
    placeholders = ', '.join('?' * len(values))
    sql = 'INSERT INTO meta ({}) VALUES ({})'.format(columns, placeholders)
    #print(sql)
    cur.execute(sql, list(values.values()))
    
    valuest = {'id': values['id'], 'text': unify.open_text(os.path.join(plain, str(values['id'])+".txt"))}
    
    columns = ', '.join(valuest.keys())
    placeholders = ', '.join('?' * len(valuest))
    sql2 = 'INSERT INTO plain_texts ({}) VALUES ({})'.format(columns, placeholders) 
    cur.execute(sql2, list(valuest.values()))
    try:
        valuest2 = {'id': values['id'], 'text': unify.open_text(open(os.path.join(tagged, str(values['id'])+".txt"),'r', encoding='utf8').read())}
        columns = ', '.join(valuest2.keys())
        placeholders = ', '.join('?' * len(valuest2))
        sql3 = 'INSERT INTO tagged_texts ({}) VALUES ({})'.format(columns, placeholders) 
        cur.execute(sql3, list(valuest2.values()))
    except:
        valuest2 = {'id': values['id'], 'text': ""}
        columns = ', '.join(valuest2.keys())
        placeholders = ', '.join('?' * len(valuest2))
        sql3 = 'INSERT INTO tagged_texts ({}) VALUES ({})'.format(columns, placeholders) 
        cur.execute(sql3, list(valuest2.values()))
    con.commit()

 55%|█████▌    | 172/311 [00:15<00:10, 12.74it/s]